In [1]:
import math
import numpy as np
from IPython.display import clear_output
from tqdm import tqdm_notebook as tqdm

import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.color_palette("bright")
import matplotlib as mpl
import matplotlib.cm as cm

import torch
from torch import Tensor
from torch import nn
from torch.nn  import functional as F 
from torch.autograd import Variable
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

import time
import random

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
def save_checkpoint(epoch, model, optimizer):
    """
    Save model checkpoint.
    :param epoch: epoch number
    :param model: model
    :param optimizer: optimizer
    """
    state = {'epoch': epoch,
             'model': model,
             'optimizer': optimizer}
    filename = 'checkpoint_resnet18_cutmix.pth.tar'
    torch.save(state, filename)

class AverageMeter(object):
    """
    Keeps track of most recent, average, sum, and count of a metric.
    """

    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [3]:
# 数据集读取

epochs = 200
pre_epoch = 0
BATCH_SIZE = 128
LR = 0.01

transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
])

train_dataset = torchvision.datasets.CIFAR10(root='data', train=True, download=False, transform=transform_train)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)

test_dataset = torchvision.datasets.CIFAR10(root='data', train=False, download=False, transform=transform_test)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=100, shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [4]:
class ResidualBlock(nn.Module):
    def __init__(self, inchannel, outchannel, stride=1):
        super(ResidualBlock, self).__init__()
        self.left = nn.Sequential(
            nn.Conv2d(inchannel, outchannel, kernel_size=3, stride=stride, padding=1, bias=False),
            nn.BatchNorm2d(outchannel),
            nn.ReLU(inplace=True),
            nn.Conv2d(outchannel, outchannel, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(outchannel)
        )
        self.shortcut = nn.Sequential()
        if stride != 1 or inchannel != outchannel:
            self.shortcut = nn.Sequential(
                nn.Conv2d(inchannel, outchannel, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(outchannel)
            )

    def forward(self, x):
        out = self.left(x)
        out += self.shortcut(x)
        out = F.relu(out)
        return out

class ResNet(nn.Module):
    def __init__(self, ResidualBlock, num_classes=10):
        super(ResNet, self).__init__()
        self.inchannel = 64
        self.conv1 = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(),
        )
        self.layer1 = self.make_layer(ResidualBlock, 64,  2, stride=1)
        self.layer2 = self.make_layer(ResidualBlock, 128, 2, stride=2)
        self.layer3 = self.make_layer(ResidualBlock, 256, 2, stride=2)
        self.layer4 = self.make_layer(ResidualBlock, 512, 2, stride=2)
        self.fc = nn.Linear(512, num_classes)

    def make_layer(self, block, channels, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)   #strides=[1,1]
        layers = []
        for stride in strides:
            layers.append(block(self.inchannel, channels, stride))
            self.inchannel = channels
        return nn.Sequential(*layers)

    def forward(self, x):
        out = self.conv1(x)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 4)    #核为4，步长为4
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out


def ResNet18():

    return ResNet(ResidualBlock)

In [5]:
def rand_bbox(size, lam):
    W = size[2]
    H = size[3]
    """1.论文里的公式2，求出B的rw,rh"""
    cut_rat = np.sqrt(1. - lam)
    cut_w = np.int(W * cut_rat)
    cut_h = np.int(H * cut_rat)
 
    # uniform
    """2.论文里的公式2，求出B的rx,ry（bbox的中心点）"""
    cx = np.random.randint(W)
    cy = np.random.randint(H)
    #限制坐标区域不超过样本大小
 
    bbx1 = np.clip(cx - cut_w // 2, 0, W)
    bby1 = np.clip(cy - cut_h // 2, 0, H)
    bbx2 = np.clip(cx + cut_w // 2, 0, W)
    bby2 = np.clip(cy + cut_h // 2, 0, H)
    """3.返回剪裁B区域的坐标值"""
    return bbx1, bby1, bbx2, bby2


In [6]:
checkpoint = 'checkpoint_resnet18_cutmix.pth.tar'
print_freq = 20

def main():
    """
    Training.
    """
    
    global start_epoch, classes, epoch, checkpoint
    
    # 初始化模型
    
    if checkpoint is None:
        start_epoch = 0
        model = ResNet18()
        optimizer = optim.SGD(model.parameters(), lr=LR, momentum=0.9, weight_decay=5e-4)
    else:
        checkpoint = torch.load(checkpoint, map_location = 'cuda')
        start_epoch = checkpoint['epoch'] + 1
        print('\nLoaded checkpoint from epoch %d.\n' % start_epoch)
        model = checkpoint['model']
        optimizer = checkpoint['optimizer']
    
    model = model.to(device)
    criterion = nn.CrossEntropyLoss()
    
    for epoch in range(start_epoch, epochs):
        
        if epoch == 100:
            optimizer = optim.SGD(model.parameters(), lr=LR*0.1, momentum=0.9, weight_decay=5e-4)
        if epoch == 150:
            optimizer = optim.SGD(model.parameters(), lr=LR*0.01, momentum=0.9, weight_decay=5e-4)
        
        train(train_loader = train_loader,
             model = model,
             criterion=criterion,
             optimizer=optimizer,
             epoch=epoch)
        save_checkpoint(epoch, model, optimizer)
        evaluate(test_loader, model)
        

def train(train_loader, model, criterion, optimizer, epoch):
    
    model = model.train()
    
    batch_time = AverageMeter()  # forward prop. + back prop. time
    data_time = AverageMeter()  # data loading time
    losses = AverageMeter()  # loss

    start = time.time()
    total = 0
    correct = 0
    alpha = 1.0
    for i, data in enumerate(train_loader, 0):
        length = len(train_loader)
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        
        if random.random()<0.5:
            lam = np.random.beta(alpha, alpha)
            rand_index = torch.randperm(inputs.size()[0]).to(device)
            labels_a = labels
            labels_b = labels[rand_index]
            bbx1, bby1, bbx2, bby2 = rand_bbox(inputs.size(), lam)
            inputs[:, :, bbx1:bbx2, bby1:bby2] = inputs[rand_index, :, bbx1:bbx2, bby1:bby2]
            lam = 1 - ((bbx2 - bbx1) * (bby2 - bby1) / (inputs.size()[-1] * inputs.size()[-2]))
        else:
            labels_a = labels
            labels_b = labels
            lam = 1.0
        optimizer.zero_grad()

        # forward + backward
        outputs = model(inputs)
        loss = criterion(outputs, labels_a)*lam + criterion(outputs, labels_b) * (1. - lam)
        loss.backward()
        optimizer.step()
        
        losses.update(loss.item(), inputs.size(0))
        batch_time.update(time.time() - start)
        start = time.time()
        
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += predicted.eq(labels.data).cpu().sum()
        if i % print_freq == 0:
            print('Epoch: [{0}][{1}/{2}]\t'
                  'Batch Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Data Time {data_time.val:.3f} ({data_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'.format(epoch, i, len(train_loader),
                                                    batch_time=batch_time,
                                                    data_time=data_time, loss=losses))
            f1 = open("train_resnet18_cutmix.txt", "a")
            f1.write('Epoch: [{0}][{1}/{2}]\t'
                  'Batch Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Data Time {data_time.val:.3f} ({data_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t\n'.format(epoch, i, len(train_loader),
                                                    batch_time=batch_time,
                                                    data_time=data_time, loss=losses))
            f1.close()
    print(correct/total)
    f2 = open("train_acc_resnet18_cutmix.txt", "a")
    f2.write('测试分类准确率为：%.3f%%\n' % (100 * correct / total))
    f2.close()
def evaluate(test_loader, model):
    
    model.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for i, data in enumerate(test_loader, 0):
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            # 取得分最高的那个类 (outputs.data的索引号)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum()
        print('测试分类准确率为：%.3f%%' % (100 * correct / total))
        f3 = open("test_acc_resnet18.txt", "a")
        f3.write('测试分类准确率为：%.3f%%\n' % (100 * correct / total))
        f3.close()
        acc = 100. * correct / total
        best_acc = 85
        if acc > best_acc:
            f3 = open("best_acc_resnet18_cutmix.txt", "w")
            f3.write("EPOCH=%d,best_acc= %.3f%%" % (epoch + 1, acc))
            f3.close()
            best_acc = acc
if __name__ == '__main__':
    main()


Loaded checkpoint from epoch 95.



<ipython-input-5-9aec3f1bf01e>:6: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  cut_w = np.int(W * cut_rat)
<ipython-input-5-9aec3f1bf01e>:7: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more detai

Epoch: [95][0/391]	Batch Time 5.267 (5.267)	Data Time 0.000 (0.000)	Loss 1.3063 (1.3063)	
Epoch: [95][20/391]	Batch Time 0.172 (0.417)	Data Time 0.000 (0.000)	Loss 1.2554 (0.5755)	
Epoch: [95][40/391]	Batch Time 0.172 (0.298)	Data Time 0.000 (0.000)	Loss 1.3415 (0.5495)	
Epoch: [95][60/391]	Batch Time 0.172 (0.257)	Data Time 0.000 (0.000)	Loss 0.0314 (0.5051)	
Epoch: [95][80/391]	Batch Time 0.173 (0.237)	Data Time 0.000 (0.000)	Loss 1.0721 (0.5062)	
Epoch: [95][100/391]	Batch Time 0.172 (0.225)	Data Time 0.000 (0.000)	Loss 0.1021 (0.5006)	
Epoch: [95][120/391]	Batch Time 0.172 (0.216)	Data Time 0.000 (0.000)	Loss 0.0781 (0.4858)	
Epoch: [95][140/391]	Batch Time 0.172 (0.210)	Data Time 0.000 (0.000)	Loss 0.0920 (0.4964)	
Epoch: [95][160/391]	Batch Time 0.172 (0.206)	Data Time 0.000 (0.000)	Loss 1.1443 (0.4906)	
Epoch: [95][180/391]	Batch Time 0.187 (0.202)	Data Time 0.000 (0.000)	Loss 0.0873 (0.4973)	
Epoch: [95][200/391]	Batch Time 0.172 (0.200)	Data Time 0.000 (0.000)	Loss 1.1449 (0.4

Epoch: [99][160/391]	Batch Time 0.172 (0.193)	Data Time 0.000 (0.000)	Loss 0.8827 (0.4082)	
Epoch: [99][180/391]	Batch Time 0.172 (0.191)	Data Time 0.000 (0.000)	Loss 0.0972 (0.4110)	
Epoch: [99][200/391]	Batch Time 0.172 (0.190)	Data Time 0.000 (0.000)	Loss 0.0393 (0.4071)	
Epoch: [99][220/391]	Batch Time 0.172 (0.189)	Data Time 0.000 (0.000)	Loss 0.0847 (0.4141)	
Epoch: [99][240/391]	Batch Time 0.172 (0.188)	Data Time 0.000 (0.000)	Loss 0.0299 (0.4090)	
Epoch: [99][260/391]	Batch Time 0.172 (0.187)	Data Time 0.000 (0.000)	Loss 0.9092 (0.4101)	
Epoch: [99][280/391]	Batch Time 0.172 (0.186)	Data Time 0.000 (0.000)	Loss 1.0927 (0.4120)	
Epoch: [99][300/391]	Batch Time 0.172 (0.186)	Data Time 0.000 (0.000)	Loss 0.0462 (0.4325)	
Epoch: [99][320/391]	Batch Time 0.187 (0.185)	Data Time 0.000 (0.000)	Loss 0.0348 (0.4343)	
Epoch: [99][340/391]	Batch Time 0.187 (0.185)	Data Time 0.000 (0.000)	Loss 1.2348 (0.4311)	
Epoch: [99][360/391]	Batch Time 0.187 (0.184)	Data Time 0.000 (0.000)	Loss 0.029

Epoch: [103][320/391]	Batch Time 0.172 (0.186)	Data Time 0.000 (0.000)	Loss 0.0127 (0.4207)	
Epoch: [103][340/391]	Batch Time 0.187 (0.186)	Data Time 0.000 (0.000)	Loss 0.0187 (0.4186)	
Epoch: [103][360/391]	Batch Time 0.182 (0.186)	Data Time 0.000 (0.000)	Loss 0.0075 (0.4079)	
Epoch: [103][380/391]	Batch Time 0.171 (0.185)	Data Time 0.000 (0.000)	Loss 0.6854 (0.4267)	
tensor(0.8757)
测试分类准确率为：95.010%
Epoch: [104][0/391]	Batch Time 2.742 (2.742)	Data Time 0.000 (0.000)	Loss 0.0051 (0.0051)	
Epoch: [104][20/391]	Batch Time 0.187 (0.301)	Data Time 0.000 (0.000)	Loss 0.0110 (0.5098)	
Epoch: [104][40/391]	Batch Time 0.172 (0.240)	Data Time 0.000 (0.000)	Loss 0.7518 (0.4680)	
Epoch: [104][60/391]	Batch Time 0.172 (0.220)	Data Time 0.000 (0.000)	Loss 0.7760 (0.5041)	
Epoch: [104][80/391]	Batch Time 0.172 (0.209)	Data Time 0.000 (0.000)	Loss 1.0286 (0.5540)	
Epoch: [104][100/391]	Batch Time 0.187 (0.203)	Data Time 0.000 (0.000)	Loss 0.0107 (0.5398)	
Epoch: [104][120/391]	Batch Time 0.187 (0.19

...

Epoch: [182][60/391]	Batch Time 0.172 (0.219)	Data Time 0.000 (0.000)	Loss 0.7129 (0.3433)	
Epoch: [182][80/391]	Batch Time 0.187 (0.209)	Data Time 0.000 (0.000)	Loss 0.0024 (0.3814)	
Epoch: [182][100/391]	Batch Time 0.172 (0.203)	Data Time 0.000 (0.000)	Loss 0.0023 (0.4006)	
Epoch: [182][120/391]	Batch Time 0.187 (0.199)	Data Time 0.000 (0.000)	Loss 0.0049 (0.3932)	
Epoch: [182][140/391]	Batch Time 0.172 (0.196)	Data Time 0.000 (0.000)	Loss 0.8979 (0.4146)	
Epoch: [182][160/391]	Batch Time 0.172 (0.194)	Data Time 0.000 (0.000)	Loss 1.2002 (0.4187)	
Epoch: [182][180/391]	Batch Time 0.187 (0.192)	Data Time 0.000 (0.000)	Loss 0.8653 (0.4185)	
Epoch: [182][200/391]	Batch Time 0.187 (0.191)	Data Time 0.000 (0.000)	Loss 0.0033 (0.4170)	
Epoch: [182][220/391]	Batch Time 0.177 (0.190)	Data Time 0.000 (0.000)	Loss 0.0021 (0.4166)	
Epoch: [182][240/391]	Batch Time 0.172 (0.189)	Data Time 0.000 (0.000)	Loss 0.0024 (0.4134)	
Epoch: [182][260/391]	Batch Time 0.172 (0.188)	Data Time 0.000 (0.000)	L

Epoch: [186][200/391]	Batch Time 0.172 (0.190)	Data Time 0.000 (0.000)	Loss 0.4992 (0.3760)	
Epoch: [186][220/391]	Batch Time 0.172 (0.189)	Data Time 0.000 (0.000)	Loss 0.7681 (0.3738)	
Epoch: [186][240/391]	Batch Time 0.187 (0.188)	Data Time 0.000 (0.000)	Loss 0.6509 (0.3738)	
Epoch: [186][260/391]	Batch Time 0.172 (0.187)	Data Time 0.000 (0.000)	Loss 0.9615 (0.3823)	
Epoch: [186][280/391]	Batch Time 0.172 (0.187)	Data Time 0.000 (0.000)	Loss 0.7580 (0.3796)	
Epoch: [186][300/391]	Batch Time 0.187 (0.186)	Data Time 0.000 (0.000)	Loss 0.6210 (0.3697)	
Epoch: [186][320/391]	Batch Time 0.172 (0.186)	Data Time 0.000 (0.000)	Loss 0.0074 (0.3651)	
Epoch: [186][340/391]	Batch Time 0.172 (0.185)	Data Time 0.000 (0.000)	Loss 0.6319 (0.3726)	
Epoch: [186][360/391]	Batch Time 0.172 (0.185)	Data Time 0.000 (0.000)	Loss 0.0016 (0.3728)	
Epoch: [186][380/391]	Batch Time 0.172 (0.185)	Data Time 0.000 (0.000)	Loss 0.8716 (0.3794)	
tensor(0.9002)
测试分类准确率为：95.880%
Epoch: [187][0/391]	Batch Time 2.709 (

Epoch: [190][340/391]	Batch Time 0.172 (0.186)	Data Time 0.000 (0.000)	Loss 0.0020 (0.4008)	
Epoch: [190][360/391]	Batch Time 0.172 (0.185)	Data Time 0.000 (0.000)	Loss 0.0018 (0.3974)	
Epoch: [190][380/391]	Batch Time 0.187 (0.185)	Data Time 0.000 (0.000)	Loss 0.9578 (0.3993)	
tensor(0.8780)
测试分类准确率为：95.880%
Epoch: [191][0/391]	Batch Time 2.718 (2.718)	Data Time 0.000 (0.000)	Loss 0.0032 (0.0032)	
Epoch: [191][20/391]	Batch Time 0.172 (0.298)	Data Time 0.000 (0.000)	Loss 0.0018 (0.4531)	
Epoch: [191][40/391]	Batch Time 0.187 (0.240)	Data Time 0.000 (0.000)	Loss 0.8822 (0.3466)	
Epoch: [191][60/391]	Batch Time 0.187 (0.219)	Data Time 0.000 (0.000)	Loss 0.0026 (0.3652)	
Epoch: [191][80/391]	Batch Time 0.172 (0.209)	Data Time 0.000 (0.000)	Loss 0.0049 (0.3822)	
Epoch: [191][100/391]	Batch Time 0.172 (0.203)	Data Time 0.000 (0.000)	Loss 0.0026 (0.3998)	
Epoch: [191][120/391]	Batch Time 0.172 (0.199)	Data Time 0.000 (0.000)	Loss 0.0019 (0.4077)	
Epoch: [191][140/391]	Batch Time 0.172 (0.19

Epoch: [195][80/391]	Batch Time 0.172 (0.209)	Data Time 0.000 (0.000)	Loss 0.3777 (0.3598)	
Epoch: [195][100/391]	Batch Time 0.187 (0.203)	Data Time 0.000 (0.000)	Loss 0.4882 (0.3714)	
Epoch: [195][120/391]	Batch Time 0.172 (0.199)	Data Time 0.000 (0.000)	Loss 0.7706 (0.3670)	
Epoch: [195][140/391]	Batch Time 0.187 (0.196)	Data Time 0.000 (0.000)	Loss 0.0024 (0.3608)	
Epoch: [195][160/391]	Batch Time 0.172 (0.194)	Data Time 0.000 (0.000)	Loss 0.0047 (0.3646)	
Epoch: [195][180/391]	Batch Time 0.172 (0.192)	Data Time 0.000 (0.000)	Loss 1.0345 (0.3742)	
Epoch: [195][200/391]	Batch Time 0.172 (0.190)	Data Time 0.000 (0.000)	Loss 0.0041 (0.3913)	
Epoch: [195][220/391]	Batch Time 0.172 (0.189)	Data Time 0.000 (0.000)	Loss 0.0021 (0.3850)	
Epoch: [195][240/391]	Batch Time 0.172 (0.188)	Data Time 0.000 (0.000)	Loss 0.7828 (0.3693)	
Epoch: [195][260/391]	Batch Time 0.172 (0.188)	Data Time 0.000 (0.000)	Loss 0.0018 (0.3821)	
Epoch: [195][280/391]	Batch Time 0.187 (0.187)	Data Time 0.000 (0.000)	

Epoch: [199][220/391]	Batch Time 0.187 (0.190)	Data Time 0.000 (0.000)	Loss 0.9901 (0.3654)	
Epoch: [199][240/391]	Batch Time 0.172 (0.189)	Data Time 0.000 (0.000)	Loss 0.0032 (0.3700)	
Epoch: [199][260/391]	Batch Time 0.172 (0.188)	Data Time 0.000 (0.000)	Loss 0.0018 (0.3702)	
Epoch: [199][280/391]	Batch Time 0.172 (0.188)	Data Time 0.000 (0.000)	Loss 0.0023 (0.3655)	
Epoch: [199][300/391]	Batch Time 0.187 (0.187)	Data Time 0.000 (0.000)	Loss 0.0056 (0.3593)	
Epoch: [199][320/391]	Batch Time 0.187 (0.187)	Data Time 0.000 (0.000)	Loss 0.0019 (0.3671)	
Epoch: [199][340/391]	Batch Time 0.187 (0.186)	Data Time 0.000 (0.000)	Loss 0.8314 (0.3742)	
Epoch: [199][360/391]	Batch Time 0.187 (0.186)	Data Time 0.000 (0.000)	Loss 1.1065 (0.3790)	
Epoch: [199][380/391]	Batch Time 0.187 (0.185)	Data Time 0.000 (0.000)	Loss 0.0034 (0.3803)	
tensor(0.8890)
测试分类准确率为：95.920%
